<a href="https://colab.research.google.com/github/hsw1805/ML-DL/blob/main/05_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 트리의 앙상블
## 랜덤 포레스트
정형 데이터에 매우 적합

데이터 뽑는 방법
- 부트스트랩 샘플
  - ex) 1000개의 샘플이 들어있는 가방에서 100개의 샘플을 뽑는다 가정
    
    1.1개의 샘플을 먼저 뽑고 다시 넣음(복원 추출)

  - 기본적으로 부트스트랩 샘플은 훈련 세트의 크기와 같게 만듦
  - 1000개의 샘플이 들어있는 가방에서 중복하여 1000개의 샘플을 뽑음



노드 분할하는 방법
- 전체 특성의 개수의 제곱근만큼의 특성을 선택
  - ex) 9개의 특성이 있다면 3개의 특성만 랜덤 선택

사이킷런의 랜덤 포레스트는 기본적으로 100개의 결정 트리를 이런 방식으로 훈련

분류일 때 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측으로 삼음



In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state=42)

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
#n_jobs = 코어 개수 몇개 쓸지
rf = RandomForestClassifier(n_jobs=-1, random_state = 42)
#교차 검증
score = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(score['train_score']),np.mean(score['test_score']))

0.9973541965122431 0.8905151032797809


In [5]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


### OOB score이란??
부트스트랩의 샘플에 포함되지 않은 데이터로 결정트리 평가

In [6]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state =42)
rf.fit(train_input, train_target)
print(rf.oob_score_)


0.8934000384837406


## 엑스트라 트리
랜덤포레스트와 비슷하게 동작하지만,부트스트랩 샘플을 사용하지 않고 전체 훈련 세트를 사용

대신 노드를 분할할 때 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할

In [9]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state = 42)
score = cross_validate(et,train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [10]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


##Gradient boosting
깊이가 앝은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방법

경사하강법을 통해 트리를 앙상블에 추가

분류에서 로지스틱 손실 함수를 사용하고 회귀에서는 평균 제곱 오차 함수를 사용

In [12]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state = 42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [13]:
#tree를 default(100개)가 아닌 500개로 하기
#오차에 대한 learning_rate는 기존 default(0.1)에서 0.2로 수정
gb = GradientBoostingClassifier(n_estimators=500, learning_rate =0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target,return_train_score=True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [14]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


##히스토그램 기반 그레이디언트 부스팅

성능이 일반적으로 더 좋음

default 값인 256으로 특성을 나누고 학습함

In [15]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state = 42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']))


/usr/local/lib/python3.10/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


0.9321723946453317


In [17]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats = 10, random_state=42, n_jobs = -1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]
